In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

In [ ]:
# Define transformation on input image and target label
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,))
])

In [ ]:
# Download and load the training data 
trainset = torchvision.datasets.EMNIST(root='./data', train=True, download=True, transform=transform, split='byclass')
train_data = [(x[0], y) for x,y in trainset if y<10] #Only keep samples from class 0-9
trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)

In [ ]:
# Download and load the test data
testset = torchvision.datasets.EMNIST(root='./data', train=False, download=True, transform=transform, split='byclass')
test_data = [(x[0], y) for x,y in testset if y<10] # Only keep samples from class 0-9
testloader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)

In [ ]:
# Defining my CNN architecture

import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
# Instatiating my model, defining my loss function and optimizer
net = Net()

net.load_state_dict(torch.load('model.pth'))
net.eval()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001) # Example learning rate

In [ ]:
# Training the model over several epochs
'''
for epoch in range(12):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0], data[1]
        
        # Add a channel dimension to inputs; transforming [64, 28, 28] to [64, 1, 28, 28]
        inputs = inputs.unsqueeze(1)  # This is the key adjustment
        
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {running_loss / (i + 1)}")
print("Finished Training")
'''


In [ ]:
# Testing the trained model
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0], data[1]

        # Add a channel dimension to images; transforming [batch_size, height, width] to [batch_size, 1, height, width]
        images = images.unsqueeze(1)  # This is the key adjustment

        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy: {100 * correct / total}%")

In [ ]:
#torch.save(net.state_dict(), 'model.pth')